# Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_feature_importances, plot_anomalies, plot_dataset

In [2]:
# Load adult data
df = pd.read_csv('data/adult.data', header=None)

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain', 
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,57,Private,298507,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,40,United-States,>50K
1,20,Private,316702,Some-college,10,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K
2,50,Private,40623,Some-college,10,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K
3,27,Private,194590,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,Black,Female,0,0,25,United-States,<=50K
4,18,Federal-gov,101709,11th,7,Never-married,Other-service,Own-child,Asian-Pac-Islander,Male,0,0,15,Philippines,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,19,Private,293928,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,20,United-States,<=50K
996,20,Private,194630,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K
997,59,Private,314149,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,1740,50,United-States,<=50K
998,34,Self-emp-not-inc,288486,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K


In [3]:
partial_features = {
    "education": {"type": "nominal"}
}

# Infer features types
features = infer_feature_attributes(df, features=partial_features)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features)


In [5]:
t.react_into_trainee(residuals=True)

accuracy = t.get_prediction_stats(stats=['accuracy'])['target'][0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.728


# Explain

How was the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Feature importance (global)

The feature importance information provides insight into the feature[s] which were primary drivers for each of the prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be extracted at the local level (regional model for each case).

In [6]:
# Extract the global MDA (mean decrease in accuracy)
t.react_into_trainee(action_feature=action_features[0], mda_robust=True, residuals=True)
global_mda = t.get_prediction_stats(action_feature=action_features[0], stats=['mda'])
plot_feature_importances(global_mda, title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

## Feature uncertainty (global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason  Howso Engine can maintain the level of performance despite noisy data is through characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. Note, the residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [7]:
# Global feature residuals
global_feature_residuals = t.get_prediction_stats(stats=['mae']).T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)
global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,79434.366494
capital-gain,1644.320836
age,11.656583
hours-per-week,8.188927
occupation,0.907231
education,0.814623
education-num,0.814623
relationship,0.715555
marital-status,0.656280
workclass,0.463741


# "Show me..."

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 



In [8]:
# Store the familiarity conviction, this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,57,Private,298507,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,40,United-States,>50K,1.399090,0,e2137036-b16c-4ec0-9a89-55717650f6ef,10.456274
1,20,Private,316702,Some-college,10,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K,1.131043,1,e2137036-b16c-4ec0-9a89-55717650f6ef,9.727783
2,50,Private,40623,Some-college,10,Divorced,Sales,Not-in-family,White,Female,0,0,40,United-States,<=50K,1.761223,2,e2137036-b16c-4ec0-9a89-55717650f6ef,11.807217
3,27,Private,194590,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,Black,Female,0,0,25,United-States,<=50K,0.956403,3,e2137036-b16c-4ec0-9a89-55717650f6ef,49.226416
4,18,Federal-gov,101709,11th,7,Never-married,Other-service,Own-child,Asian-Pac-Islander,Male,0,0,15,Philippines,<=50K,0.349602,4,e2137036-b16c-4ec0-9a89-55717650f6ef,73.650080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,19,Private,293928,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,0,0,20,United-States,<=50K,0.728085,995,e2137036-b16c-4ec0-9a89-55717650f6ef,7.263543
996,20,Private,194630,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K,0.442983,996,e2137036-b16c-4ec0-9a89-55717650f6ef,4.764119
997,59,Private,314149,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,1740,50,United-States,<=50K,0.723027,997,e2137036-b16c-4ec0-9a89-55717650f6ef,54.322125
998,34,Self-emp-not-inc,288486,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.698722,998,e2137036-b16c-4ec0-9a89-55717650f6ef,8.160336


In [9]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.75

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [10]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,21,Private,334618,Some-college,10,Never-married,Protective-serv,Not-in-family,Black,Female,99999,0,40,United-States,>50K,0.074864,386,e2137036-b16c-4ec0-9a89-55717650f6ef,182.084514,outlier
1,32,Private,170154,Assoc-acdm,12,Separated,Exec-managerial,Unmarried,White,Female,25236,0,50,United-States,>50K,0.142306,338,e2137036-b16c-4ec0-9a89-55717650f6ef,119.650491,outlier
2,64,Private,251292,5th-6th,3,Separated,Other-service,Other-relative,White,Female,0,0,20,Cuba,<=50K,0.174243,969,e2137036-b16c-4ec0-9a89-55717650f6ef,106.140199,outlier
3,32,Self-emp-not-inc,180303,Bachelors,13,Divorced,Craft-repair,Unmarried,Asian-Pac-Islander,Male,0,0,47,Iran,<=50K,0.194136,793,e2137036-b16c-4ec0-9a89-55717650f6ef,99.814908,outlier
4,30,Private,117747,HS-grad,9,Married-civ-spouse,Sales,Wife,Asian-Pac-Islander,Female,0,1573,35,?,<=50K,0.201693,491,e2137036-b16c-4ec0-9a89-55717650f6ef,97.712945,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,65,?,79272,Some-college,10,Widowed,?,Not-in-family,Asian-Pac-Islander,Female,0,0,6,United-States,<=50K,0.672882,291,e2137036-b16c-4ec0-9a89-55717650f6ef,55.793564,outlier
69,44,Self-emp-inc,359259,HS-grad,9,Divorced,Craft-repair,Not-in-family,White,Male,0,0,60,Portugal,<=50K,0.685061,988,e2137036-b16c-4ec0-9a89-55717650f6ef,55.426293,outlier
70,56,Private,295067,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Male,14084,0,45,United-States,>50K,0.710745,396,e2137036-b16c-4ec0-9a89-55717650f6ef,50.536023,outlier
71,59,Private,314149,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,0,1740,50,United-States,<=50K,0.723027,997,e2137036-b16c-4ec0-9a89-55717650f6ef,54.322125,outlier


In [11]:
# Cache global non-robust residuals into trainee
t.react_into_trainee(residuals=True)

# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {'robust_computation': True,
           'global_case_feature_residual_convictions': True, 
           'local_case_feature_residual_convictions': True}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [12]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['details']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['details']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [13]:
plot_anomalies(outliers, local_case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an outlier. The darker the shade of red, the higher the contribution to the case being an outlier. 

## Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [14]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,19,Private,200136,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K,0.142469,235,e2137036-b16c-4ec0-9a89-55717650f6ef,0.713692,inlier
1,19,Private,224241,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K,0.161600,61,e2137036-b16c-4ec0-9a89-55717650f6ef,0.927985,inlier
2,18,Private,192409,Some-college,10,Never-married,Sales,Own-child,White,Female,0,0,20,United-States,<=50K,0.172499,582,e2137036-b16c-4ec0-9a89-55717650f6ef,1.073296,inlier
3,23,Private,198996,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K,0.198236,717,e2137036-b16c-4ec0-9a89-55717650f6ef,1.550564,inlier
4,24,Private,152189,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K,0.208020,596,e2137036-b16c-4ec0-9a89-55717650f6ef,1.652727,inlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,19,?,28967,Some-college,10,Never-married,?,Not-in-family,White,Male,0,0,40,United-States,<=50K,0.733420,343,e2137036-b16c-4ec0-9a89-55717650f6ef,9.042955,inlier
188,43,Private,112131,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,<=50K,0.734989,673,e2137036-b16c-4ec0-9a89-55717650f6ef,7.228241,inlier
189,46,Private,102359,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K,0.744539,232,e2137036-b16c-4ec0-9a89-55717650f6ef,7.128270,inlier
190,62,?,129246,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,0,0,40,United-States,<=50K,0.745458,736,e2137036-b16c-4ec0-9a89-55717650f6ef,7.631429,inlier


In [15]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [16]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['details']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['details']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [17]:
plot_anomalies(inliers, local_case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an inlier. The darker the shade of blue, the higher the contribution the to case being an inlier.

## Potential improvements

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

In [18]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]


In [19]:
# Residual convictions are output via the local_case_feature_residual_convictions explanation
details = {'global_case_feature_residual_convictions':True}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(), 
                     leave_case_out=True, 
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(), 
                     action_features=action_features,
                     details=details)

In [20]:
# Extract residual conviction
target_residual_convictions = [ x['target'] for x in new_result['details']['global_case_feature_residual_convictions'] ]

# Binarize residual conviction
convict_threshold = 0.75
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [21]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,57,Private,298507,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,40,United-States,>50K,1.399090,0,e2137036-b16c-4ec0-9a89-55717650f6ef,10.456274,0.095636,0.399481,1
1,20,Private,316702,Some-college,10,Never-married,Prof-specialty,Own-child,White,Male,...,20,United-States,<=50K,1.131043,1,e2137036-b16c-4ec0-9a89-55717650f6ef,9.727783,0.102798,1.580392,0
2,50,Private,40623,Some-college,10,Divorced,Sales,Not-in-family,White,Female,...,40,United-States,<=50K,1.761223,2,e2137036-b16c-4ec0-9a89-55717650f6ef,11.807217,0.084694,5.582766,0
3,27,Private,194590,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,Black,Female,...,25,United-States,<=50K,0.956403,3,e2137036-b16c-4ec0-9a89-55717650f6ef,49.226416,0.020314,14.900144,0
4,18,Federal-gov,101709,11th,7,Never-married,Other-service,Own-child,Asian-Pac-Islander,Male,...,15,Philippines,<=50K,0.349602,4,e2137036-b16c-4ec0-9a89-55717650f6ef,73.650080,0.013578,2.459638,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,19,Private,293928,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Female,...,20,United-States,<=50K,0.728085,995,e2137036-b16c-4ec0-9a89-55717650f6ef,7.263543,0.137674,0.892529,0
996,20,Private,194630,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,...,40,United-States,<=50K,0.442983,996,e2137036-b16c-4ec0-9a89-55717650f6ef,4.764119,0.209902,24.472008,0
997,59,Private,314149,Assoc-voc,11,Married-civ-spouse,Sales,Husband,White,Male,...,50,United-States,<=50K,0.723027,997,e2137036-b16c-4ec0-9a89-55717650f6ef,54.322125,0.018409,3.335427,0
998,34,Self-emp-not-inc,288486,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,<=50K,0.698722,998,e2137036-b16c-4ec0-9a89-55717650f6ef,8.160336,0.122544,2.381080,0


In [22]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()
    
    m = (max_size - min_size) / (max_value - min_value)
    
    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

In [23]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  
